# Chapter 6: Maintaining your Delta Lake
> The following exercises use the New York Times [Covid-19 NYT Dataset](https://github.com/delta-io/delta-docs/tree/main/static/quickstart_docker/rs/data/COVID-19_NYT).

The dataset can be found in the `delta_quickstart` docker.

In [7]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import col, desc, to_date
from delta.tables import DeltaTable

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS default.covid_nyt (
  date DATE
) USING DELTA
TBLPROPERTIES('delta.logRetentionDuration'='interval 7 days');
""")


In [ ]:
spark.sql("show tables").show()

In [ ]:
# will be empty on the first run. this is expected
len(spark.table("default.covid_nyt").inputFiles())

In [ ]:
# uncomment if you'd like to begin again
#spark.sql("drop table default.covid_nyt")

## Start Populating the Table
> The next three commands are used to show Schema Evolution and Validation with Delta Lake

In [ ]:
# Populate the Table reading the Parquet covid_nyc Data
# note: this will fail on the first run, and that is okay
(spark.read
      .format("parquet")
      .load("/opt/spark/work-dir/rs/data/COVID-19_NYT/*.parquet")
      .withColumn("date", to_date("date", "yyyy-MM-dd"))
      .write
      .format("delta")
      .saveAsTable("default.covid_nyt"))

In [ ]:
# one step closer, there is still something missing...
# and yes, this operation still fails... if only...
(spark.read
      .format("parquet")
      .load("/opt/spark/work-dir/rs/data/COVID-19_NYT/*.parquet")
      .withColumn("date", to_date("date", "yyyy-MM-dd"))
      .write
      .format("delta")
      .mode("append")
      .saveAsTable("default.covid_nyt"))

## Schema Evolution: Handle Automatically
If you trust the upstream data source (provider) then you can add the `option("mergeSchema", "true")`. Otherwise, it is better to specifically select a subset of the columns you expected to see. In this example use case, the only known column is `date`, so it is fairly safe to power ahead.

In [ ]:
# Evolve the Schema. (Showcases how to auto-merge changes to the schema)
# note: if you can trust the upstream, then this option is perfectly fine
# however, if you don't trust the upstream, then it is good to opt-in to the 
# changing columns.

(spark.read
      .format("parquet")
      .load("/opt/spark/work-dir/rs/data/COVID-19_NYT/*.parquet")
      .withColumn("date", to_date("date", "yyyy-MM-dd"))
      .write
      .format("delta")
      .mode("append")
      .option("mergeSchema", "true")
      .saveAsTable("default.covid_nyt")
    )

In [ ]:
df = spark.table("default.covid_nyt")
df.count()

# Alternatives to Auto Schema Evolution
In the previous case, we used `.option("mergeSchema", "true")` to modify the behavior of the Delta Lake writer. While this option simplifies how we evolve our Delta Lake table schemas, it comes at the price of not being fully aware of the changes to our table schema. In the case where there are unknown columns being introduced from an upstream source, you'll want to know which columns are intended to bring forward, and which columns can be safely ignored.

## Intentionally Adding Columns with Alter Table

In [ ]:
# manually set the columns. This is an example of intentional opt-in to the new columns outside of '.option("mergeSchema", "true")`. 
# Note: this can be run once, afterwards the ADD columns will fail since they already exist
spark.sql("""
ALTER TABLE default.covid_nyt 
ADD columns (
  county STRING,
  state STRING,
  fips INT,
  cases INT,
  deaths INT
);
""")
# notice how we are only using `.mode("append")` and explicitly add `.option("mergeSchema", "false")`. 
# this is how we stop unwanted columns from being freely added to our Delta Lake tables. It comes at the cost of raising exceptions and failing the job.
# a failed job might seem like a bad option, but it is the cheaper option since you are intentionally blocking unknown data from flowing into your tables. 
(spark.read
      .format("parquet")
      .load("/opt/spark/work-dir/rs/data/COVID-19_NYT/*.parquet")
      .withColumn("date", to_date("date", "yyyy-MM-dd"))
      .write
      .format("delta")
      .option("mergeSchema", "false")
      .mode("append")
      .saveAsTable("default.covid_nyt"))

In [78]:
spark.sql("describe extended default.covid_nyt").show(truncate=False)

+----------------------------+----------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                     |comment|
+----------------------------+----------------------------------------------------------------------------------------------+-------+
|date                        |date                                                                                          |       |
|county                      |string                                                                                        |       |
|state                       |string                                                                                        |       |
|fips                        |int                                                                                           |       |
|cases                       |int                             

In [79]:
spark.sql("select * from default.covid_nyt limit 10").show(truncate=True)

+----------+----------+--------+-----+-----+------+
|      date|    county|   state| fips|cases|deaths|
+----------+----------+--------+-----+-----+------+
|2020-05-19|  Lawrence|Illinois|17101|    4|     0|
|2020-05-19|       Lee|Illinois|17103|   75|     1|
|2020-05-19|Livingston|Illinois|17105|   27|     1|
|2020-05-19|     Logan|Illinois|17107|   10|     0|
|2020-05-19|     Macon|Illinois|17115|  170|    17|
|2020-05-19|  Macoupin|Illinois|17117|   42|     1|
|2020-05-19|   Madison|Illinois|17119|  499|    45|
|2020-05-19|    Marion|Illinois|17121|   49|     0|
|2020-05-19|  Marshall|Illinois|17123|    5|     0|
|2020-05-19|     Mason|Illinois|17125|   16|     0|
+----------+----------+--------+-----+-----+------+



# Adding and Modifying Table Properties

In [93]:
spark.sql("""
  ALTER TABLE default.covid_nyt 
  SET TBLPROPERTIES (
    'catalog.team_name'='dldg_authors',
    'catalog.engineering.comms.slack'='https://delta-users.slack.com/archives/CG9LR6LN4',
    'catalog.engineering.comms.email'='dldg_authors@gmail.com',
    'catalog.table.classification'='all-access'
  )""")


DataFrame[]

In [ ]:
# view the table history
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, 'default.covid_nyt')
dt.history(10).select("version", "timestamp", "operation").show()

In [ ]:
# use DeltaTable to view
dt.detail().select("properties").show(truncate=False)

In [ ]:
# view the table properties
spark.sql("show tblproperties default.covid_nyt").show(truncate=False)

## Removing Table Properties

In [ ]:
# add incorrect table property
# which is blocked by default
spark.conf.set("spark.databricks.delta.allowArbitraryProperties.enabled","true")
# now we can make a mistake
spark.sql("""
  ALTER TABLE default.covid_nyt 
  SET TBLPROPERTIES (
    'delta.loRgetentionDuratio'='interval 7 days'
  )""")

In [98]:
# luckily, we can remove the unwanted table property using UNSET
spark.sql("""
  ALTER TABLE default.covid_nyt 
  UNSET TBLPROPERTIES ('delta.loRgetentionDuratio')
""")
# now that we are done, let's just add back the safe guard again
spark.conf.set("spark.databricks.delta.allowArbitraryProperties.enabled","false")

## Delta Table Optimization

In [ ]:
## Creating the Small File Problem

from delta.tables import DeltaTable
(DeltaTable.createIfNotExists(spark)
    .tableName("default.nonoptimal_covid_nyt")
    .property("description", "table to be optimized")
    .property("catalog.team_name", "dldg_authors")
    .property("catalog.engineering.comms.slack",
	"https://delta-users.slack.com/archives/CG9LR6LN4")
    .property("catalog.engineering.comms.email","dldg_authors@gmail.com")
    .property("catalog.table.classification","all-access")
    .addColumn("date", "DATE")
    .addColumn("county", "STRING")
    .addColumn("state", "STRING")
    .addColumn("fips", "INT")
    .addColumn("cases", "INT")
    .addColumn("deaths", "INT")
    .execute())

In [ ]:
#spark.sql("drop table default.nonoptimal_covid_nyt")

In [ ]:
(spark
   .table("default.covid_nyt")
   .repartition(9000)
   .write
   .format("delta")
   .mode("overwrite")
   .saveAsTable("default.nonoptimal_covid_nyt")
)

## Using Optimize to Fix the Small Files Problem

In [144]:
(
    DeltaTable.forName(spark, "default.nonoptimal_covid_nyt")
    .optimize()
    .executeCompaction()
)

23/06/07 06:46:52 WARN TaskSetManager: Stage 618 contains a task of very large size (1584 KiB). The maximum recommended task size is 1000 KiB.


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

## Cleaning up our Delta Tables using Vacuum

In [145]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","false")
DeltaTable.forName(spark, "default.nonoptimal_covid_nyt").vacuum(retentionHours=0)
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","true")

Deleted 9000 files and directories in a total of 1 directories.
